In [1]:
import pandas as pd

full_data_file = r'C:\Users\RoopakPrajapat\Downloads\35b0ee6d670f438a9c73486fff6fd627\35b0ee6d670f438a9c73486fff6fd627'
concern_keys = pd.read_excel(r"C:\Users\RoopakPrajapat\Downloads\concern_keys.xlsx")
non_concern_keys = pd.read_excel(r"C:\Users\RoopakPrajapat\Downloads\non_concern_keys.xlsx")

In [2]:
import dask.dataframe as dd
full_data_df = dd.read_parquet(full_data_file)
# full_data_df = full_data_df.compute()
full_data_df.head()

,VendorNumber,CompanyCode,PhysicalPaymentNo,SystemPaymentNo,SystemInvoiceNo,InvoiceLineRef,DebitCreditIndicator,FiscalYear,DocumentType,CheckNumber,...,FMP2P1060_Score,FMP2P1061_Score,FMP2P1062_Score,Previous_Overall_Risk_Score,Previous_Overall_Tran_Risk_Score,Overall_Risk_Score,Overall_Tran_Risk_Score,Tests_Failed,Tests_Failed_Count,Tests_FailedID
0,0100019625,US10,62311,2000078293,5100285871,001,H,2022,RE-Invoice - Gross,,...,0.0,0.0,0.0,None,None,0.00,0.00,<NA>,0,NaN
1,0100019625,US10,63649,2000086568,5100320114,001,H,2022,RE-Invoice - Gross,,...,0.0,0.0,0.0,None,None,0.00,0.00,<NA>,0,NaN
2,0100019625,US10,63303,2000083054,5100310191,001,H,2022,RE-Invoice - Gross,,...,0.0,0.0,0.0,None,None,0.00,0.00,<NA>,0,NaN
3,0100019625,US10,63497,2000085400,5100315737,001,H,2022,RE-Invoice - Gross,,...,0.0,0.0,0.0,None,None,0.00,0.00,<NA>,0,NaN
4,0100000258,US10,8000088275,2000055621,5100182975,001,H,2021,RE-Invoice - Gross,,...,0.0,0.0,0.0,None,None,51.37,47.08,FMP2P039: Duplicate payments that have the sam...,2,44541.0


In [3]:
index_col = 'SPT_RowID'
concern_keys = concern_keys[index_col]
non_concern_keys = non_concern_keys[index_col]

In [4]:
amount_column = 'AmountExcl_OC'

In [5]:
full_data_df['target'] = None

# full_data_df.loc[full_data_df[index_col].isin(concern_keys), 'target'] = 1
# full_data_df.loc[full_data_df[index_col].isin(non_concern_keys), 'target'] = 0

def set_column_value_if_exists(df, column_to_check, column_to_set, value_list, new_value):
    df.loc[df[column_to_check].isin(value_list), column_to_set] = new_value
    return df

full_data_df = full_data_df.map_partitions(set_column_value_if_exists, column_to_check=index_col, column_to_set='target', value_list=concern_keys, new_value=1)
full_data_df = full_data_df.map_partitions(set_column_value_if_exists, column_to_check=index_col, column_to_set='target', value_list=non_concern_keys, new_value=0)

In [6]:
print("Number of concern records: ", len(concern_keys))
print("Number of non concern records: ", len(non_concern_keys))
print('Total records in data', len(full_data_df))

Number of concern records:  17
Number of non concern records:  540
Total records in data 563549


In [12]:
from typing import Any
import numpy as np
from ipywidgets import interact
import matplotlib.pyplot as plt


# Get the first digits of the number
test_digits = 2
def first_digits(value: Any):
    try:
        number = float(value)

        # should not be less than 10 ** (test_digits - 1)
        if abs(number) < 10 ** (test_digits - 1):
            return int(abs(number))

        # take only the number before the decimal point and mark absolute value
        number_str = str(abs(number)).split('.')[0]
        return int(number_str[:test_digits])
    except Exception:
        return 0
    
def calc_std_error(prob, n):
    return np.sqrt(prob * (1 - prob) / n)

def display_matplotlib_benford_graph(df):
    fig, ax1 = plt.subplots(figsize=(15, 6))
    ax1.bar(df['first_digits'], df['first_digits_frequency'], color='skyblue', label='Frequency')
    ax1.bar(df[df['benford_anomaly'] == True]['first_digits'], df[df['benford_anomaly'] == True]['first_digits_frequency'], color='red', label='Anomalies')


    # show frequency as data labels on top of the bars
    # for i, v in enumerate(df['first_digits_frequency']):
    #     ax1.text(i + 1, v + 0.01, str(v), color='black', ha='center')

    # Creating secondary y-axis
    ax2 = ax1.twinx()
    # Add actual probability
    ax2.plot(df['first_digits'], df['first_digits_probs'], color='black', label='Actual Probability')
    ax2.plot(df['first_digits'], df['benford_probs'], color='blue', label='Benford Probability')
    ax2.plot(df['first_digits'], df['upper_bound'], color='red', label='Upper Bound')
    # ax2.plot(df['first_digits'], df['lower_bound'], color='red', label='Lower Bound')
    plt.show()

def calc_benford_anomaly(s, return_anomaly=False):
    df = pd.DataFrame()
    df[amount_column] = s
    df.loc[:,'first_digits'] = df[amount_column].apply(first_digits)
    df.loc[:,'first_digits_frequency'] = df['first_digits'].map(df['first_digits'].value_counts())
    df.loc[:,'first_digits_probs'] = df['first_digits_frequency'] / len(df)
    df.loc[:,'benford_probs'] = df['first_digits'].apply(lambda x: np.log10(1 + 1/x) if x > 0 else 0)
    df.loc[:,'std_error'] = df.apply(lambda x: calc_std_error(x['benford_probs'], x['first_digits_frequency']), axis=1)
    df.loc[:,'upper_bound'] = df['benford_probs'] + 2.57 * df['std_error']
    df.loc[:,'lower_bound'] = df['benford_probs'] - 2.57 * df['std_error']
    # df.loc[:,'benford_anomaly'] = (df['first_digits_probs'] < df['lower_bound']) | (df['first_digits_probs'] > df['upper_bound'])
    df.loc[:,'benford_anomaly'] = (df['first_digits_probs'] > df['upper_bound'])
    if return_anomaly:
        return df['benford_anomaly']
    
    return df

def plot_calc_benford_anomaly(df):
    if not isinstance(df, pd.DataFrame):
        df = df[[amount_column, 'target']].compute()
    else:
        df = df[[amount_column, 'target']]

    anomaly_df = calc_benford_anomaly(df[amount_column])
    anomaly_df['target'] = df['target']

    print('Total Benford Anomalies for Concerns: ')
    display(anomaly_df[anomaly_df['target'] == 1]['benford_anomaly'].value_counts())
    print('Total Benford Anomalies for Non Concerns: ')
    display(anomaly_df[anomaly_df['target'] == 0]['benford_anomaly'].value_counts())
    
    graph_cols = ['first_digits','first_digits_frequency', 'first_digits_probs','benford_probs', 'upper_bound', 'lower_bound','benford_anomaly']
    if not anomaly_df.empty:
        anomaly_df = anomaly_df[anomaly_df['first_digits'] > 9][graph_cols].drop_duplicates()
        display_matplotlib_benford_graph(anomaly_df.sort_values('first_digits'))
        

In [9]:
from itertools import combinations
analysis_cols = ['VendorNumber', 'PaymentMethod', 'RemitCountry', 'GLAccount']

combs = []
for i in range(1, len(analysis_cols) + 1):
    for comb in combinations(analysis_cols, i):
        combs.append(comb)

combs_data = full_data_df[analysis_cols + [amount_column, 'target']].compute()
for combination in combs:
    column_name = '_'.join(combination) + '_BenAnomaly'
    grouped = combs_data.groupby(list(combination))[amount_column]
    combs_data[column_name] = (grouped.transform(lambda x : calc_benford_anomaly(x, return_anomaly=True)))

# all anomaly columns
anomaly_cols = [col for col in combs_data.columns if 'BenAnomaly' in col]

# calculate sum by column
combs_columns = []
for col in anomaly_cols:
    combs_columns.append((col, combs_data[col].sum()))
# sort by sum in descending order
combs_columns = sorted(combs_columns, key=lambda x: x[1], reverse=True)
display(combs_columns)

# calculate sum by row
combs_data['Total Anomalies'] = combs_data[anomaly_cols].sum(axis=1)
display(combs_data.sort_values('Total Anomalies', ascending=False))

combs_data.to_excel('benford_anomaly_combinations.xlsx', index=False)

[('VendorNumber_PaymentMethod_GLAccount_BenAnomaly', 135877),
 ('VendorNumber_PaymentMethod_RemitCountry_GLAccount_BenAnomaly', 135877),
 ('VendorNumber_PaymentMethod_BenAnomaly', 135866),
 ('VendorNumber_PaymentMethod_RemitCountry_BenAnomaly', 135866),
 ('VendorNumber_RemitCountry_BenAnomaly', 129497),
 ('VendorNumber_BenAnomaly', 129495),
 ('VendorNumber_RemitCountry_GLAccount_BenAnomaly', 129401),
 ('VendorNumber_GLAccount_BenAnomaly', 129399),
 ('PaymentMethod_RemitCountry_GLAccount_BenAnomaly', 67923),
 ('PaymentMethod_RemitCountry_BenAnomaly', 67338),
 ('PaymentMethod_GLAccount_BenAnomaly', 65845),
 ('PaymentMethod_BenAnomaly', 65088),
 ('RemitCountry_BenAnomaly', 64753),
 ('RemitCountry_GLAccount_BenAnomaly', 56020),
 ('GLAccount_BenAnomaly', 54233)]

,VendorNumber,PaymentMethod,RemitCountry,GLAccount,AmountExcl_OC,target,VendorNumber_BenAnomaly,PaymentMethod_BenAnomaly,RemitCountry_BenAnomaly,GLAccount_BenAnomaly,...,VendorNumber_GLAccount_BenAnomaly,PaymentMethod_RemitCountry_BenAnomaly,PaymentMethod_GLAccount_BenAnomaly,RemitCountry_GLAccount_BenAnomaly,VendorNumber_PaymentMethod_RemitCountry_BenAnomaly,VendorNumber_PaymentMethod_GLAccount_BenAnomaly,VendorNumber_RemitCountry_GLAccount_BenAnomaly,PaymentMethod_RemitCountry_GLAccount_BenAnomaly,VendorNumber_PaymentMethod_RemitCountry_GLAccount_BenAnomaly,Total Anomalies
61817,0100019004,-,US,0021201010,210.00,None,True,True,True,True,...,True,True,True,True,True,True,True,True,True,15
69947,0100019004,-,US,0021201010,210.00,None,True,True,True,True,...,True,True,True,True,True,True,True,True,True,15
69937,0100019004,-,US,0021201010,210.00,None,True,True,True,True,...,True,True,True,True,True,True,True,True,True,15
69938,0100019004,-,US,0021201010,210.00,None,True,True,True,True,...,True,True,True,True,True,True,True,True,True,15
69939,0100019004,-,US,0021201010,210.00,None,True,True,True,True,...,True,True,True,True,True,True,True,True,True,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5402,0100003260,6-ACH (CTX),US,0021201010,2600.00,None,False,False,False,False,...,False,False,False,False,False,False,False,False,False,0
5403,0100003260,6-ACH (CTX),US,0021201010,3800.00,None,False,False,False,False,...,False,False,False,False,False,False,False,False,False,0
97494,0100019856,6-ACH (CTX),US,0021201010,9246.67,None,False,False,False,False,...,False,False,False,False,False,False,False,False,False,0
97493,0100019856,6-ACH (CTX),US,0021201010,9250.53,None,False,False,False,False,...,False,False,False,False,False,False,False,False,False,0


In [13]:
from IPython.display import clear_output

@interact
def display_anomalies(
    VendorNumber=['All'] + sorted(full_data_df['VendorNumber'].unique().compute().tolist()),
    PaymentMethod=['All'] + sorted(full_data_df['PaymentMethod'].unique().compute().tolist()),
    RemitCountry=['All'] + sorted(full_data_df['RemitCountry'].unique().compute().tolist()),
    GLAccount=['All'] + sorted(full_data_df['GLAccount'].unique().compute().tolist())
):
    clear_output(wait=True)

    filtered_data = full_data_df.copy()
    if VendorNumber != 'All':
        filtered_data = filtered_data.loc[filtered_data['VendorNumber'] == VendorNumber]
    if PaymentMethod != 'All':
        filtered_data = filtered_data.loc[filtered_data['PaymentMethod'] == PaymentMethod]
    if RemitCountry != 'All':
        filtered_data = filtered_data.loc[filtered_data['RemitCountry'] == RemitCountry]
    if GLAccount != 'All':
        filtered_data = filtered_data.loc[filtered_data['GLAccount'] == GLAccount]

    filtered_data = plot_calc_benford_anomaly(filtered_data)

interactive(children=(Dropdown(description='VendorNumber', options=('All', '0000012202', '0000012325', '000001…